In [1]:
import os

In [2]:
# get current working directory
os.getcwd()

'c:\\projects_endtoend\\Chest-X-Ray-Multi-Class-Classifier\\notebook'

In [3]:
# to get inside project folder/directory
os.chdir("../")

In [4]:
os.getcwd() 

'c:\\projects_endtoend\\Chest-X-Ray-Multi-Class-Classifier'

## Update the entity

In [5]:
from pathlib import Path
from dataclasses import dataclass

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir : Path
    source_url : str
    local_data_file : Path
    unzip_dir : Path

## Update the configuration manager in src config

In [6]:
from cnn_classifier.constants import *
from cnn_classifier.utils.utils import read_yaml, create_directories

In [7]:
class ConfigurationManager:

    def __init__(self, config_filepath = CONFIG_FILE_PATH, params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)  ## reading config.yaml file 
        self.params = read_yaml(params_filepath)  ## reading params.yaml file

        create_directories([self.config.artifacts_root])  

    ## data ingestion related config
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion    

        create_directories([config.root_dir])  

        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,          
            source_url  = config.source_url,      
            local_data_file = config.local_data_file,   
            unzip_dir = config.unzip_dir         
        ) 


        return data_ingestion_config



## Update the Components

In [8]:
import os
import zipfile
import gdown
from cnn_classifier import logger
from src.cnn_classifier.utils.utils import get_size

In [9]:
class DataIngestion:

    def __init__(self, config : DataIngestionConfig):
        self.config = config

    
    def download_file(self) -> str:
        " fetch data from the url "

        try:

            dataset_url = self.config.source_url  
            zip_download_dir = self.config.local_data_file  
            os.makedirs("artifacts/data_ingestion",exist_ok = True) 

            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")

            file_id = dataset_url.split("/")[-2]
            prefix = 'https://drive.google.com/uc?/export=download&id='
            gdown.download(prefix + file_id, zip_download_dir)  

            logger.info(f'Downloaded data from {dataset_url} into file {zip_download_dir}')


        except Exception as e:
            raise e


    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir      
        os.makedirs(unzip_path, exist_ok=True)  

        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref: 
            zip_ref.extractall(unzip_path)      

## Update the Pipeline

In [10]:
try:

    config = ConfigurationManager()   
    data_ingestion_config = config.get_data_ingestion_config()   
    data_ingestion = DataIngestion(config=data_ingestion_config) 
    data_ingestion.download_file()     
    data_ingestion.extract_zip_file()  

except Exception as e:
    raise e

[2025-12-05 20:26:07,604: INFO: utils: yaml file: config\config.yaml loaded successfully]
[2025-12-05 20:26:07,608: INFO: utils: yaml file: params.yaml loaded successfully]
[2025-12-05 20:26:07,610: INFO: utils: created directory at: artifacts]
[2025-12-05 20:26:07,610: INFO: utils: created directory at: artifacts/data_ingestion]
[2025-12-05 20:26:07,614: INFO: 3515868962: Downloading data from https://drive.google.com/file/d/1lLbSqACVv2eC_kjKyqCLvy9Rn4y2qx7N/view?usp=sharing into file artifacts/data_ingestion/data.zip]


Downloading...
From (original): https://drive.google.com/uc?/export=download&id=1lLbSqACVv2eC_kjKyqCLvy9Rn4y2qx7N
From (redirected): https://drive.google.com/uc?%2Fexport=download&id=1lLbSqACVv2eC_kjKyqCLvy9Rn4y2qx7N&confirm=t&uuid=f7c6987e-f58a-48fa-8761-3ae068896629
To: c:\projects_endtoend\Chest-X-Ray-Multi-Class-Classifier\artifacts\data_ingestion\data.zip
100%|██████████| 291M/291M [01:00<00:00, 4.82MB/s] 

[2025-12-05 20:27:12,040: INFO: 3515868962: Downloaded data from https://drive.google.com/file/d/1lLbSqACVv2eC_kjKyqCLvy9Rn4y2qx7N/view?usp=sharing into file artifacts/data_ingestion/data.zip]
